# Using large language models for exploratory data analysis

The following Google Colab notebook implements an simple exploratory data analysis with Langchain, Azure's OpenAI GPT-4 on a SQLite3's northwind database.

### Install Libraries

In [1]:
!pip3 install --quiet langchain openai langchain_experimental langchain-openai SQLAlchemy


### Download the Northwind database for SQLite3

In [2]:
!wget https://tdmdal.github.io/mma-sql-2021/data/northwind.sqlite3

--2024-01-21 13:22:35--  https://tdmdal.github.io/mma-sql-2021/data/northwind.sqlite3
Resolving tdmdal.github.io (tdmdal.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to tdmdal.github.io (tdmdal.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 804864 (786K) [application/octet-stream]
Saving to: ‘northwind.sqlite3.1’

northwind.sqlite3.1 100%[===================>] 786.00K  --.-KB/s    in 0.05s   

2024-01-21 13:22:35 (15.4 MB/s) - ‘northwind.sqlite3.1’ saved [804864/804864]



### Load the SQL magic extension

[this extension](https://github.com/catherinedevlin/ipython-sql) allows us to connect to DBs and issue SQL command


In [3]:
%load_ext sql

### Load and test the database

In [4]:
%sql sqlite:///northwind.sqlite3

In [5]:
%%sql
SELECT COUNT(OrderID) FROM Orders WHERE type='table' AND name NOT LIKE 'sqlite_%';

 * sqlite:///northwind.sqlite3
(sqlite3.OperationalError) no such column: type
[SQL: SELECT COUNT(OrderID) FROM Orders WHERE type='table' AND name NOT LIKE 'sqlite_%';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [6]:
import os
import logging
import sys
import textwrap
import pprint
import pandas as pd
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import AgentType, create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from sqlalchemy import create_engine
import getpass
import json
import re
import warnings
import importlib
import ast
import matplotlib.pyplot as plt

### Enter Azure OpenAI credentials

In [7]:
print('Enter AZURE_OPENAI_API_KEY:')
os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass()

Enter AZURE_OPENAI_API_KEY:
··········


In [8]:
print('Enter AZURE_OPENAI_ENDPOINT:')
os.environ["AZURE_OPENAI_ENDPOINT"] = getpass.getpass()

Enter AZURE_OPENAI_ENDPOINT:
··········


### Ask the database

In [9]:
chat_llm = AzureChatOpenAI(
    temperature=0,
    openai_api_version="2023-05-15",
    deployment_name="gpt-4",
)

In [10]:
odbc_str = "sqlite:///northwind.sqlite3"
db_engine = create_engine(odbc_str)

In [11]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
          You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about Categories, Products and Orders.
         """
         ),
        ("user", "{question}\n ai: "),
    ]
)

In [12]:
db = SQLDatabase(db_engine)

sql_toolkit = SQLDatabaseToolkit(db=db, llm=chat_llm)
sql_toolkit.get_tools()

sqldb_agent = create_sql_agent(
    llm=chat_llm,
    toolkit=sql_toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [13]:
sqldb_agent.run(final_prompt.format(
        question="How many Orders are there?"
  ))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Categories, Customers, Employees, OrderDetails, Orders, Products, Shippers, Suppliers
Thought:The Orders table seems to be the most relevant for this query. I should check its schema to understand its structure.
Action: sql_db_schema
Action Input: "Orders" 
Observation: 
CREATE TABLE "Orders" (
	"OrderID" INTEGER NOT NULL, 
	"CustomerID" NCHAR(5) NOT NULL, 
	"EmployeeID" INTEGER, 
	"OrderDate" DATETIME, 
	"RequiredDate" DATETIME, 
	"ShippedDate" DATETIME, 
	"ShipVia" INTEGER, 
	"Freight" NUMERIC DEFAULT 0, 
	"ShipName" NVARCHAR(40), 
	"ShipAddress" NVARCHAR(60), 
	"ShipCity" NVARCHAR(15), 
	"ShipRegion" NVARCHAR(15), 
	"ShipPostalCode" NVARCHAR(10), 
	"ShipCountry" NVARCHAR(15), 
	PRIMARY KEY ("OrderID"), 
	FOREIGN KEY("ShipVia") REFERENCES "Shippers" ("ShipperID"), 
	FOREIGN KEY("EmployeeID") REFERENCES "Employees" ("EmployeeID"), 
	FOREIGN KEY("CustomerID") REFERENCES "Customers" ("Custom

'There are 830 orders.'